<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Mathias González Espinoza.
- Nombre de alumno 2: Eduardo Javier Silva Gaete.


## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** [https://github.com/Zerzer0/MDS7202](https://github.com/Zerzer0/MDS7202)

# Importamos librerias útiles

In [1]:
!pip install xgboost
!pip install optuna

                                              0.0/99.8 MB ? eta -:--:--
                                              0.1/99.8 MB 2.9 MB/s eta 0:00:35
                                              0.4/99.8 MB 3.8 MB/s eta 0:00:27
                                              0.4/99.8 MB 3.8 MB/s eta 0:00:27
                                              0.5/99.8 MB 2.3 MB/s eta 0:00:43
                                              0.6/99.8 MB 2.6 MB/s eta 0:00:38
                                              0.9/99.8 MB 3.1 MB/s eta 0:00:33
                                              1.1/99.8 MB 3.3 MB/s eta 0:00:31
                                              1.4/99.8 MB 3.4 MB/s eta 0:00:29
                                              1.6/99.8 MB 3.5 MB/s eta 0:00:29
                                              1.7/99.8 MB 3.5 MB/s eta 0:00:28
                                              1.7/99.8 MB 3.5 MB/s eta 0:00:28
                                              1.7/99.8 MB 3


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/409.6 kB ? eta -:--:--
     --------                                92.2/409.6 kB 1.7 MB/s eta 0:00:01
     -------------------------------        337.9/409.6 kB 3.0 MB/s eta 0:00:01
     -------------------------------        337.9/409.6 kB 3.0 MB/s eta 0:00:01
     -------------------------------------  399.4/409.6 kB 1.9 MB/s eta 0:00:01
     -------------------------------------  399.4/409.6 kB 1.9 MB/s eta 0:00:01
     -------------------------------------  399.4/409.6 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 409.6/409.6 kB 1.2 MB/s eta 0:00:00
                                              0.0/226.8 kB ? eta -:--:--
     -------------------------------------  225.3/226.8 kB 6.9 MB/s eta 0:00:01
     -------------------------------------  225.3/226.8 kB 6.9 MB/s eta 0:00:01
     -------------------------------------- 226.8/226.8 kB 2.3 MB/s eta 0:00:00
                                              0.0/2.1 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
camelot 12.6.29 requires SQLAlchemy<0.8.0,>=0.7.7, but you have sqlalchemy 2.0.23 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7456 entries, 0 to 7455
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         7456 non-null   int64         
 1   date       7456 non-null   datetime64[ns]
 2   city       7456 non-null   object        
 3   lat        7456 non-null   float64       
 4   long       7456 non-null   float64       
 5   pop        7456 non-null   int64         
 6   shop       7456 non-null   object        
 7   brand      7456 non-null   object        
 8   container  7456 non-null   object        
 9   capacity   7456 non-null   object        
 10  price      7456 non-null   float64       
 11  quantity   7456 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(3), object(5)
memory usage: 699.1+ KB


In [3]:
df[['lat', 'long', 'pop', 'price', 'quantity']].corr()

,lat,long,pop,price,quantity
lat,1.000000,-0.715204,0.068339,0.001782,0.000896
long,-0.715204,1.000000,0.220037,-0.002226,-0.151079
pop,0.068339,0.220037,1.000000,0.000370,-0.214428
price,0.001782,-0.002226,0.000370,1.000000,-0.518616
quantity,0.000896,-0.151079,-0.214428,-0.518616,1.000000


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import joblib

df[['lat', 'long']].astype('category')
df[["city", "shop", "brand", "container", "capacity"]] = df[["city", "shop", "brand", "container", "capacity"]].apply(lambda col: col.astype('category'))
#df[["city", "shop", "brand", "container", "capacity"]].astype("category")

# 1
train_data, test_data = train_test_split(df.drop(columns=['id']), test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

X_train, y_train = train_data.drop('quantity', axis=1), train_data['quantity']
X_val, y_val = val_data.drop('quantity', axis=1), val_data['quantity']
X_test, y_test = test_data.drop('quantity', axis=1), test_data['quantity']

# 2
def extraccion_fecha(df):
    df['dia'] = df['date'].dt.day.astype('category')
    df['mes'] = df['date'].dt.month.astype('category')
    df['año'] = df['date'].dt.year.astype('category')
    return df.drop('date', axis=1)

date_transformer = FunctionTransformer(extraccion_fecha)

# 3
columnas_numericas = ['price', 'pop']
columnas_categoricas = ['dia', 'mes', 'año', 'lat', 'long', 'city', 'shop', 'brand', 'container', 'capacity']


preprocessor = ColumnTransformer(
    transformers=[
        ('StandarScaler', StandardScaler(), columnas_numericas),
        ('OneHotEncoder', OneHotEncoder(sparse_output=False), columnas_categoricas)
    ])

# 4
pipeline_dummy = Pipeline([
    ('date_transform', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy='mean'))
])

# 5
pipeline_dummy.fit(X_train, y_train)
y_pred_dummy = pipeline_dummy.predict(X_val)
mae_dummy = mean_absolute_error(y_val, y_pred_dummy)
print(f"MAE Dummy Regressor: {mae_dummy}")

# 6
pipeline_xgb = Pipeline([
    ('date_transform', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

pipeline_xgb.fit(X_train, y_train)
y_pred_xgb = pipeline_xgb.predict(X_val)
mae_xgb = mean_absolute_error(y_val, y_pred_xgb)
print(f"MAE XGB Regressor: {mae_xgb}")

# 7
joblib.dump(pipeline_dummy, 'dummy_regressor_model.pkl')
joblib.dump(pipeline_xgb, 'xgb_regressor_model.pkl')

MAE Dummy Regressor: 13576.56699051175
MAE XGB Regressor: 2494.8668331140557


['xgb_regressor_model.pkl']

**Respuesta:** En el contexto del negocio, el MAE representa cuánto, en promedio, el modelo se equivoca al predecir la cantidad de ventas.

Comparando los resultados de XGBRegressor con el del DummyRegressor, podemos notar una clara disminución, por lo tanto podemos decir que implementar XGBRegressor tuvo mejores resultados.

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [17]:
preprocessor.set_output(transform="pandas")
preprocessor.transform(extraccion_fecha(df))

,StandarScaler__price,StandarScaler__pop,OneHotEncoder__dia_28,OneHotEncoder__dia_29,OneHotEncoder__dia_30,OneHotEncoder__dia_31,OneHotEncoder__mes_1,OneHotEncoder__mes_2,OneHotEncoder__mes_3,OneHotEncoder__mes_4,...,OneHotEncoder__brand_gazoza,OneHotEncoder__brand_kinder-cola,OneHotEncoder__brand_lemon-boost,OneHotEncoder__brand_orange-power,OneHotEncoder__container_can,OneHotEncoder__container_glass,OneHotEncoder__container_plastic,OneHotEncoder__capacity_1.5lt,OneHotEncoder__capacity_330ml,OneHotEncoder__capacity_500ml
0,-0.290054,1.374306,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2.038835,1.374306,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,-0.400370,1.374306,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,-0.241025,1.374306,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,-0.988720,1.374306,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7451,1.622086,1.339417,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
7452,1.205338,1.339417,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
7453,-0.424884,-0.801230,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
7454,1.193080,0.002598,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [18]:
pipeline_xgb_monotonic = Pipeline([
    ('date_transform', date_transformer),
    ('preprocessor', preprocessor),
    # Para nuestro caso la primera columna del df es la variable 'price'
    ('regressor', XGBRegressor(monotone_constraints='(-1)'))
])

pipeline_xgb_monotonic.fit(X_train, y_train)
y_pred_xgb_monotonic = pipeline_xgb_monotonic.predict(X_val)
mae_xgb_monotonic = mean_absolute_error(y_val, y_pred_xgb_monotonic)
print("MAE (XGBRegressor con restricción monótona):", mae_xgb_monotonic)

joblib.dump(pipeline_xgb_monotonic, 'xgb_regressor_monotone_model.pkl')

MAE (XGBRegressor con restricción monótona): 2617.0149860865313


['xgb_regressor_monotone_model.pkl']

**Respuesta:** Al ver el resultado, podemos notar un incremento en el MAE respecto al no definir la relación, es decir, un peor resultado, por lo que podemos decir que nuestro amigo estaba equivocado.

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

In [69]:
import optuna
from optuna.samplers import TPESampler

seed = 42

sampler = TPESampler(seed=seed)

def objective(trial):
    params_xgb = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
    }

    params_ohe = {
        'min_frequency': trial.suggest_float('min_frecuency', 0.0, 1.0)
    }

    preprocessor_optuna = ColumnTransformer(
        transformers=[
            ('StandarScaler', StandardScaler(), columnas_numericas),
            ('OneHotEncoder', OneHotEncoder(sparse_output=False, **params_ohe), columnas_categoricas)
        ])
    
    pipeline_optuna = Pipeline([
        ('date_transform', date_transformer),
        ('preprocessor', preprocessor_optuna),
        ('regressor', XGBRegressor(random_state=seed, **params_xgb))
    ])

    pipeline_optuna.fit(X_train, y_train)
    y_pred = pipeline_optuna.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)

    return mae

# Configurar el estudio de optimización
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, timeout=300)

# Obtener los mejores hiperparámetros
best_params_optuna = study.best_params
print("Mejores hiperparámetros encontrados:")
for key in best_params_optuna:
    print(f"{key}:{best_params_optuna[key]}")

# Crear el pipeline final con los mejores hiperparámetros
pipeline_optimized = Pipeline([
    ('date_transform', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(random_state=seed, **best_params_optuna))
])

pipeline_optimized.fit(X_train, y_train)

y_pred_optimized = pipeline_optimized.predict(X_val)
mae_optimized = mean_absolute_error(y_val, y_pred_optimized)
print("MAE con hiperparámetros optimizados:", mae_optimized)

joblib.dump(pipeline_optimized, 'optimized_model.pkl')

[I 2023-11-19 23:11:02,115] A new study created in memory with name: no-name-bd8ef648-af9f-4486-ae2d-804b4f9fef7e


[I 2023-11-19 23:11:15,055] Trial 0 finished with value: 9488.779513727179 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 954, 'max_depth': 8, 'max_leaves': 60, 'min_child_weight': 1, 'reg_alpha': 0.15599452033620265, 'reg_lambda': 0.05808361216819946, 'min_frecuency': 0.8661761457749352}. Best is trial 0 with value: 9488.779513727179.
[I 2023-11-19 23:11:18,757] Trial 1 finished with value: 5817.137573606034 and parameters: {'learning_rate': 0.06051038616257767, 'n_estimators': 723, 'max_depth': 3, 'max_leaves': 97, 'min_child_weight': 5, 'reg_alpha': 0.21233911067827616, 'reg_lambda': 0.18182496720710062, 'min_frecuency': 0.18340450985343382}. Best is trial 1 with value: 5817.137573606034.
[I 2023-11-19 23:11:22,381] Trial 2 finished with value: 8893.201936315318 and parameters: {'learning_rate': 0.03111998205299424, 'n_estimators': 549, 'max_depth': 6, 'max_leaves': 29, 'min_child_weight': 4, 'reg_alpha': 0.13949386065204183, 'reg_lambda': 0.29214464853521815

Mejores hiperparámetros encontrados:
learning_rate:0.05643230081593082
n_estimators:742
max_depth:8
max_leaves:41
min_child_weight:2
reg_alpha:0.4266623946411109
reg_lambda:0.7439140220122968
min_frecuency:0.03668047155635063


c:\Users\Eduardo S\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:160: UserWarning:

[23:16:12] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "min_frecuency" } are not used.




MAE con hiperparámetros optimizados: 2159.490485246775


['optimized_model.pkl']

**Respuesta:**

En primer lugar, se comenzará definiendo cada hiperparametro optimizado. Para el caso del modelo XGBRegressor, estos son:

- learning_rate: Este parámetro controla la contribución de cada árbol al modelo general. Un valor más bajo implica un menor aporte de cada arbol, además de generar un modelo más preciso evitando el sobreajuste. Es por lo anterior que optimizar este hiperparametro entre (0.001, 0.1) tiene bastante sentido.

- n_estimators: Corresponde al número de árboles que se crearán en el entrenamiento. Mientras mayor sea la cantidad de arboles, mayor será el modelo, sin embargo una cantidad excesiva puede generar sobreajuste en el modelo. Por lo tanto, un rango entre (50, 1000) es razonable.

- max_depth: Representa la profundidad de un arbol. Un valor más alto permite que los árboles sean más complejos y se ajusten mejor a los datos de entrenamiento, pero también puede aumentar el riesgo de sobreajuste. Por tanto un rango entre (3, 10) es razonable.

- max_leaves: Representa el número máximo de nodos hoja en cada árbol. Se elige un rango entre (0, 100) dado que numeros más altos podrian generar sobreajuste.

- min_child_weight: Representa la suma mínima de pesos de observaciones necesaria en una hoja. Se elige entre (1, 5) dado que valores más altos producen sobreajuste. 

- reg_alpha: Término de regularización L1 en los pesos de la hoja. Añade una penalización proporcional al valor absoluto de los pesos de la hoja. Solo existen dos posibles valores, 0 y 1 para determinar si se aplica o no este hiperparametro.

- reg_lambda: Término de regularización L2 en los pesos de la hoja. Añade una penalización proporcional al cuadrado de los pesos de la hoja. Solo existen dos posibles valores, 0 y 1 para determinar si se aplica o no este hiperparametro.


Para el caso del modelo OneHotEncoder, el hiperparametro a optimizar es: 

- min_frecuency: Especifica la frecuencia mínima requerida para que una categoría sea considerada. Debe ser un valor en el rango [0.0, 1.0], donde 0.0 significa que todas las categorías se deben incluir, y 1.0 significa que solo las categorías que aparecen  los datos de entrenamiento se incluirán.

Los mejores Hiperparametros encontrados en 300 trials son:

-learning_rate:0.06278382127299323

-n_estimators:964

-max_depth:8

-max_leaves:38

-min_child_weight:2

-reg_alpha:0.4679912472989679

-reg_lambda:0.5617518357559627

-min_frecuency:0.058847861045626365

Con un MAE igual a 2136.74619362571.

Comparadolo con los modelos anteriores, este dio los mejores resultados en terminos de MAE. Esto se debe a la optimización de los Hiperparametros con Optuna, acción que no fue realizada con los casos anteriores, en los cuales se utilizaron hiperparametros por defecto.







## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [55]:
# Inserte su código acá
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
from optuna.integration import XGBoostPruningCallback

seed = 42

sampler = TPESampler(seed=seed)


def objective_prunners(trial):
    params_xgb = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
    }

    params_ohe = {
        'min_frequency': trial.suggest_float('min_frecuency', 0.0, 1.0)
    }

    preprocessor_optuna = ColumnTransformer(
        transformers=[
            ('StandarScaler', StandardScaler(), columnas_numericas),
            ('OneHotEncoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore',**params_ohe), columnas_categoricas)
        ])


     
    
    pipeline_optuna_1 = Pipeline([
        ('date_transform', date_transformer),
        ('preprocessor', preprocessor_optuna)
        #('regressor', XGBRegressor(random_state=seed, **params_xgb))
    ])

    pipeline_optuna_2 = Pipeline([
        ('date_transform', date_transformer),
        ('preprocessor', preprocessor_optuna),
        ('regressor', XGBRegressor(random_state=seed, **params_xgb))
    ])

    pruning_callback = XGBoostPruningCallback(trial, 'validation_1-mae')

    pipeline_optuna_1.fit(X_train, y_train)

    X_val_transformed = pipeline_optuna_1.transform(X_val)
    X_train_transformed = pipeline_optuna_1.transform(X_train)

    print(X_val_transformed)

    pipeline_optuna_2.fit(
        X_train, y_train,
        regressor__eval_metric='mae',  # Establece la métrica que se utilizará para prunning
        regressor__eval_set=[(X_train_transformed, y_train), (X_val_transformed, y_val)],  # Datos de validación
        regressor__callbacks=[pruning_callback]  # Callback de prunning
    )
    

    

    y_pred = pipeline_optuna_2.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)

    
    

    return mae

# Configurar el estudio de optimización con pruner
#study = optuna.create_study(direction='minimize', sampler=sampler, pruner=pruner)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective_prunners, timeout=300)

# Obtener los mejores hiperparámetros
best_params_optuna = study.best_params
print("Mejores hiperparámetros encontrados:")
for key in best_params_optuna:
    print(f"{key}:{best_params_optuna[key]}")

# Crear el pipeline final con los mejores hiperparámetros
pipeline_optimized = Pipeline([
    ('date_transform', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(random_state=seed, **best_params_optuna, enable_categorical=True))
])

pipeline_optimized.fit(X_train, y_train)
y_pred_optimized = pipeline_optimized.predict(X_val)
mae_optimized = mean_absolute_error(y_val, y_pred_optimized)
print("MAE con hiperparámetros optimizados:", mae_optimized)

joblib.dump(pipeline_optimized, 'optimized_model_prunners.pkl')



[I 2023-11-19 19:15:07,758] A new study created in memory with name: no-name-3714da73-a0b9-405b-bf01-df0f57887107


[[-0.42488424  1.35735791  1.         ...  1.          1.
   1.        ]
 [-0.43714155  1.34397487  1.         ...  1.          1.
   1.        ]
 [-0.02039307  1.34397487  1.         ...  1.          1.
   1.        ]
 ...
 [ 2.51686972  1.35735791  1.         ...  1.          1.
   1.        ]
 [-0.71905964 -0.03101278  1.         ...  1.          1.
   1.        ]
 [ 0.06540808 -0.9134562   1.         ...  1.          1.
   1.        ]]
[0]	validation_0-mae:13160.52775	validation_1-mae:13295.93712
[1]	validation_0-mae:12879.89269	validation_1-mae:13035.21272
[2]	validation_0-mae:12614.36267	validation_1-mae:12790.22169
[3]	validation_0-mae:12363.37642	validation_1-mae:12555.12596
[4]	validation_0-mae:12126.84076	validation_1-mae:12326.81689
[5]	validation_0-mae:11903.46858	validation_1-mae:12117.85985


c:\Users\Eduardo S\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\Eduardo S\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[6]	validation_0-mae:11693.29371	validation_1-mae:11918.35401
[7]	validation_0-mae:11494.23572	validation_1-mae:11727.81016
[8]	validation_0-mae:11304.66664	validation_1-mae:11546.74707
[9]	validation_0-mae:11126.59153	validation_1-mae:11376.28394
[10]	validation_0-mae:10958.72646	validation_1-mae:11213.05730
[11]	validation_0-mae:10802.20584	validation_1-mae:11063.63257
[12]	validation_0-mae:10655.82707	validation_1-mae:10918.04171
[13]	validation_0-mae:10515.72612	validation_1-mae:10781.99636
[14]	validation_0-mae:10382.27901	validation_1-mae:10650.30276
[15]	validation_0-mae:10256.70621	validation_1-mae:10530.18760
[16]	validation_0-mae:10141.03958	validation_1-mae:10415.68785
[17]	validation_0-mae:10031.22022	validation_1-mae:10311.20059
[18]	validation_0-mae:9927.33292	validation_1-mae:10211.79081
[19]	validation_0-mae:9829.60381	validation_1-mae:10117.46960
[20]	validation_0-mae:9736.81448	validation_1-mae:10030.51807
[21]	validation_0-mae:9650.42521	validation_1-mae:9947.39602
[

[I 2023-11-19 19:15:55,311] Trial 0 finished with value: 9488.779513727179 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 954, 'max_depth': 8, 'max_leaves': 60, 'min_child_weight': 1, 'reg_alpha': 0.15599452033620265, 'reg_lambda': 0.05808361216819946, 'min_frecuency': 0.8661761457749352}. Best is trial 0 with value: 9488.779513727179.


...

[I 2023-11-19 19:20:18,068] Trial 33 finished with value: 2481.4665438201496 and parameters: {'learning_rate': 0.08657745641882802, 'n_estimators': 327, 'max_depth': 5, 'max_leaves': 18, 'min_child_weight': 5, 'reg_alpha': 0.5043416368074808, 'reg_lambda': 0.5159713162314986, 'min_frecuency': 0.06881511667973472}. Best is trial 15 with value: 2395.6062821275964.


Mejores hiperparámetros encontrados:
learning_rate:0.09985926473211934
n_estimators:434
max_depth:7
max_leaves:17
min_child_weight:4
reg_alpha:0.49814244393577495
reg_lambda:0.42527661711425313
min_frecuency:0.0024159981506190363


c:\Users\Eduardo S\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:160: UserWarning: [19:20:18] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "min_frecuency" } are not used.

  warnings.warn(smsg, UserWarning)


MAE con hiperparámetros optimizados: 2395.6062821275964


['optimized_model_prunners.pkl']

Pruning hace referencia a la poda de hojas de un arbol de un modelo de machine learning. La poda de hojas de utiliza con el fin de eliminar partes no esenciales de un arbol, y con ello aumenat la eficiencia del modelo e impedir el sobreajuste en los datos de entrenamiento.

Respecto a la optimización como tal, los mejores Hiperparametros encontrados en 300 trials son:

Mejores hiperparámetros encontrados:

-learning_rate:0.09985926473211934

-n_estimators:434

-max_depth:7

-max_leaves:17

-min_child_weight:4

-reg_alpha:0.49814244393577495

-reg_lambda:0.42527661711425313

-min_frecuency:0.0024159981506190363

Con un MAE igual a 2395.6062821275964.

Comparadolo con el modelo anterior, este dio peores resultados en terminos de MAE. Lo anterior puede deberse debido a que la poda de arboles esta eliminando partes esenciales del modelo, haciendolo más ineficiente que cuando no se aplica pruning.



## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [56]:
# Inserte su código acá

from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline



In [57]:
plot_optimization_history(study)

Observando el grafico anterior, se aprecia que desde el trial 15 el MAE se mantiene constante, representando el modelo con los mejores hiperparametros. Por otra parte, se observa que existen disminuciones considerables en el trial 0 y 12.

In [58]:
plot_parallel_coordinate(study)

Observando el grafico de coordenadas paralelas, se aprecia una clara relación positiva entre los hiperparametros "learning_rate" y "min_child_weight", que coinciden con los casos en los cuales el MAE es cercano a su valor minimo óptimo. Por ende, es claro que para obtener un buen rendimiento en el modelo, estos dos hiperparametros deben ser altos.

In [59]:
plot_param_importances(study)

En el gráfico anterior se observa que los hiperparametros "max_leaves" y "min_frecuency" son, por lejos, los más determinantes al momento de optimizar el modelo. Luego les sigue el hiperparametro "learning_rate" con un 0.14 de importancia.

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento? 

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [67]:
dic_MAE = {
    'dummy_regressor_model': 13576.566,
    'xgb_regressor_monotone_model': 2617.014,
    'xgb_regressor_model': 2494.866,
    'optimized_model': 2136.746 ,
    'optimized_model_prunners': 2395.606
}





df_MAE = pd.DataFrame([dic_MAE])
df_MAE = df_MAE.rename(index={0: 'MAE'})

In [68]:
df_MAE

,dummy_regressor_model,xgb_regressor_monotone_model,xgb_regressor_model,optimized_model,optimized_model_prunners
MAE,13576.566,2617.014,2494.866,2136.746,2395.606


En la tabla anterior se aprecia que el modelo con menor MAE es el "optimized_model", el cual corresponde al modelo con los hiperparametros optimizados sin pruning

In [71]:
import pickle


model_optimized = pickle.load(open('optimized_model.pkl','rb'))

In [73]:
y_pred_optimized = pipeline_optimized.predict(X_test)
mae_optimized = mean_absolute_error(y_test, y_pred_optimized)

print(f"MAE del conjunto de test: {mae_optimized}")

MAE del conjunto de test: 2026.4818485045241


Se aprecia que el modelo al ser evaluado bajo el conjunto de test, se obtiene un MAE menor al obtenido al evaluarlo en el conjunto de validación (MAE: 2136.74). Lo anterior puede ocurrir debido a multiples factores. Es posible que debido a la variabilidad de los datos en el conjunto de test, además de ser un conjunto más pequeño que el de validación, el modelo haya predicho mejor el conjunto señalado. Por otra parte, es factible que los hiperparametros utilizados se adapten mejor para el conjunto de test que el de validación.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>